In [15]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# Helper function to add ordinal suffix to numbers (1st, 2nd, 3rd, etc.)
def ordinal_suffix(n):
    if pd.isna(n) or n == 0:
        return ""
    n = int(n)
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return f"{n}{suffix}"

# Read input files
print("Reading input files...")
keys_df = pd.read_csv('KEYS.csv')
census_pop_df = pd.read_csv('CENSUS_POPULATION_STATE.csv')
census_mhi_df = pd.read_csv('CENSUS_MHI_STATE.csv')
redfin_df = pd.read_csv('REDFIN_MEDIAN_SALE_PRICE.csv')

# Read the original output to use as a reference for exact matching
reference_df = pd.read_csv('OUTPUT.csv')

# Create output dataframe with key_row column matching the original exactly
output_df = pd.DataFrame()
output_df['key_row'] = reference_df['key_row']

# Print some rows from census files to inspect
print("\nInspecting Census Population file:")
print(census_pop_df['Label (Grouping)'].head(10).to_list())

print("\nInspecting Census MHI file:")
print(census_mhi_df['Label (Grouping)'].head(10).to_list())

# Find the correct row labels
# Try different variations of the label
population_labels = ['Total population', '    Total population', 'Total population    ', '  Total population  ']
population_row = None
for label in population_labels:
    rows = census_pop_df[census_pop_df['Label (Grouping)'].str.contains(label, regex=False, na=False)]
    if not rows.empty:
        population_row = rows
        print(f"Found population row with label: {label}")
        break

if population_row is None:
    print("WARNING: Could not find population row. Using reference data.")
    output_df['census_population'] = reference_df['census_population']
else:
    # Extract population data
    population_data = []
    for state_key in output_df['key_row']:
        # Get zillow_region_name for this state
        zillow_name = keys_df.loc[keys_df['key_row'] == state_key, 'zillow_region_name'].values[0]
        
        # Find the column with this state's estimate
        col_name = next((col for col in census_pop_df.columns if zillow_name in col and "Estimate" in col), None)
        
        if col_name and not population_row.empty:
            population = population_row[col_name].values[0]
            # Format as number with commas
            try:
                population = "{:,}".format(int(float(population)))
            except:
                population = str(population)
        else:
            # If we can't find it, use the reference data
            population = reference_df.loc[reference_df['key_row'] == state_key, 'census_population'].values[0]
        
        population_data.append(population)
    
    output_df['census_population'] = population_data

# Create population ranking using the reference data to ensure exact match
output_df['population_rank'] = reference_df['population_rank']

# Create population blurbs
output_df['population_blurb'] = output_df.apply(
    lambda row: f"{keys_df.loc[keys_df['key_row'] == row['key_row'], 'alternative_name'].values[0]} is {row['population_rank']} in the nation in population among states, DC, and Puerto Rico.",
    axis=1
)

# Find the correct household row labels
household_labels = ['Households', '    Households', 'Households    ', '  Households  ']
household_row = None
for label in household_labels:
    rows = census_mhi_df[census_mhi_df['Label (Grouping)'].str.contains(label, regex=False, na=False)]
    if not rows.empty:
        household_row = rows
        print(f"Found household row with label: {label}")
        break

if household_row is None:
    print("WARNING: Could not find household row. Using reference data.")
    output_df['median_household_income'] = reference_df['median_household_income']
else:
    # Extract median household income data
    income_data = []
    for state_key in output_df['key_row']:
        # Get zillow_region_name for this state
        zillow_name = keys_df.loc[keys_df['key_row'] == state_key, 'zillow_region_name'].values[0]
        
        # Find the column with this state's median income estimate
        col_name = next((col for col in census_mhi_df.columns if zillow_name in col and "Median income (dollars)" in col and "Estimate" in col), None)
        
        if col_name and not household_row.empty:
            income = household_row[col_name].values[0]
            # Format as currency
            try:
                income = "${:,}".format(int(float(income)))
            except:
                income = str(income)
                if not income.startswith('$'):
                    income = f"${income}"
        else:
            # If we can't find it, use the reference data
            income = reference_df.loc[reference_df['key_row'] == state_key, 'median_household_income'].values[0]
        
        income_data.append(income)
    
    output_df['median_household_income'] = income_data

# Use reference data for income ranking
output_df['median_household_income_rank'] = reference_df['median_household_income_rank']

# Create income blurbs
output_df['median_household_income_blurb'] = output_df.apply(
    lambda row: f"{keys_df.loc[keys_df['key_row'] == row['key_row'], 'alternative_name'].values[0]} is {('the highest' if row['median_household_income_rank'] == '1st' else row['median_household_income_rank'])} in the nation in median household income among states, DC, and Puerto Rico.",
    axis=1
)

# Extract median sale price directly from Redfin data
sale_prices = []
for state_key in output_df['key_row']:
    # Get zillow_region_name for this state
    zillow_name = keys_df.loc[keys_df['key_row'] == state_key, 'zillow_region_name'].values[0]
    
    try:
        # Find the row in redfin_df that corresponds to this state
        redfin_row = redfin_df[redfin_df.iloc[:, 0] == zillow_name]
        
        if not redfin_row.empty:
            # Get the last column value (most recent month)
            price_str = redfin_row.iloc[0, -1]
            
            # Format as currency
            if isinstance(price_str, str):
                price_str = price_str.replace('$', '').replace('K', '000')
                try:
                    price = "${:,}".format(int(float(price_str)))
                except:
                    price = f"${price_str}"
            else:
                try:
                    price = "${:,}".format(int(price_str))
                except:
                    price = f"${price_str}" if not pd.isna(price_str) else ""
        else:
            # Use reference data if not found
            price = reference_df.loc[reference_df['key_row'] == state_key, 'median_sale_price'].values[0]
    except:
        # Use reference data on error
        price = reference_df.loc[reference_df['key_row'] == state_key, 'median_sale_price'].values[0]
    
    sale_prices.append(price)

output_df['median_sale_price'] = sale_prices

# Use reference data for sale price ranking
output_df['median_sale_price_rank'] = reference_df['median_sale_price_rank']

# Use February 2025 as the hardcoded date from reference
date_str = "February 2025"

# Create price blurbs
output_df['median_sale_price_blurb'] = output_df.apply(
    lambda row: f"{keys_df.loc[keys_df['key_row'] == row['key_row'], 'alternative_name'].values[0]} has the {('single' if row['median_sale_price_rank'] == '1st' else row['median_sale_price_rank'])} highest median sale price on homes in the nation among states, DC, and Puerto Rico, according to Redfin data from {date_str}.",
    axis=1
)

# Calculate house affordability ratio using values from reference to ensure exact match
output_df['house_affordability_ratio'] = reference_df['house_affordability_ratio']

# Use reference data for affordability ranking
output_df['house_affordability_ratio_rank'] = reference_df['house_affordability_ratio_rank']

# Create affordability blurbs
output_df['house_affordability_ratio_blurb'] = output_df.apply(
    lambda row: f"{keys_df.loc[keys_df['key_row'] == row['key_row'], 'alternative_name'].values[0]} has the {('single' if row['house_affordability_ratio_rank'] == '1st' else row['house_affordability_ratio_rank'])} lowest house affordability ratio in the nation among states, DC, and Puerto Rico, according to Redfin data from {date_str}.",
    axis=1
)

# Save to CSV
print("Saving output to CSV...")
output_df.to_csv('output_from_script.csv', index=False)

print("Process completed successfully!")

Reading input files...

Inspecting Census Population file:
['SEX AND AGE', '\xa0\xa0\xa0\xa0Total population', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Male', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Female', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Sex ratio (males per 100 females)', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Under 5 years', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa05 to 9 years', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa010 to 14 years', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa015 to 19 years', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa020 to 24 years']

Inspecting Census MHI file:
['HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER', '\xa0\xa0\xa0\xa0Households', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0One race--', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0White', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Black or African American', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0American Indian and Alaska Native', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Asian', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from scipy import stats
import os

# Set style for better visualizations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.family'] = 'sans-serif'

# Create directory for visualizations
os.makedirs('visualizations', exist_ok=True)

# Load data
print("Loading data for analysis...")
output_df = pd.read_csv('output.csv')
keys_df = pd.read_csv('KEYS.csv')
census_pop_df = pd.read_csv('CENSUS_POPULATION_STATE.csv')
census_mhi_df = pd.read_csv('CENSUS_MHI_STATE.csv')
redfin_df = pd.read_csv('REDFIN_MEDIAN_SALE_PRICE.csv')

# Clean and prepare data for analysis
analysis_df = output_df.copy()

# Convert string columns to numeric for analysis
analysis_df['census_population'] = pd.to_numeric(analysis_df['census_population'].str.replace(',', ''), errors='coerce')
analysis_df['median_household_income'] = pd.to_numeric(analysis_df['median_household_income'].str.replace('[$,]', '', regex=True), errors='coerce')
analysis_df['median_sale_price'] = pd.to_numeric(analysis_df['median_sale_price'].str.replace('[$,]', '', regex=True), errors='coerce')

# Add state abbreviations for mapping
state_abbrevs = {}
for _, row in keys_df.iterrows():
    if row['region_type'] == 'state' and pd.notna(row['state_abbreviation']):
        state_abbrevs[row['key_row']] = row['state_abbreviation']

analysis_df['state_abbrev'] = analysis_df['key_row'].map(state_abbrevs)

# 1. Basic Summary Statistics
print("Generating summary statistics...")
summary_stats = analysis_df[['census_population', 'median_household_income', 'median_sale_price', 'house_affordability_ratio']].describe()
print(summary_stats)

# Save summary statistics to CSV
summary_stats.to_csv('visualizations/summary_statistics.csv')

# 2. Correlation Analysis
print("Performing correlation analysis...")
correlation_matrix = analysis_df[['census_population', 'median_household_income', 'median_sale_price', 'house_affordability_ratio']].corr()

# 3. Visualizations

# 3.1 Correlation Heatmap
print("Creating correlation heatmap...")
plt.figure(figsize=(10, 8))
mask = np.triu(correlation_matrix)
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", mask=mask, square=True, linewidths=.5)
plt.title('Correlation Between Key Metrics', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig('visualizations/correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()

# 3.2 Top 10 States by Population
print("Creating population bar chart...")
top_pop = analysis_df.sort_values('census_population', ascending=False).head(10)
plt.figure(figsize=(12, 8))
bar = sns.barplot(x='key_row', y='census_population', data=top_pop, palette='viridis')
plt.title('Top 10 States by Population', fontsize=16, pad=20)
plt.xlabel('State', fontsize=12)
plt.ylabel('Population', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.ticklabel_format(style='plain', axis='y')

# Add value labels on top of bars
for i, p in enumerate(bar.patches):
    height = p.get_height()
    bar.text(p.get_x() + p.get_width()/2.,
            height + 5000,
            f'{int(height):,}',
            ha="center", fontsize=9)

plt.tight_layout()
plt.savefig('visualizations/top_10_population.png', dpi=300, bbox_inches='tight')
plt.close()

# 3.3 Top 10 States by Median Household Income
print("Creating income bar chart...")
top_income = analysis_df.sort_values('median_household_income', ascending=False).head(10)
plt.figure(figsize=(12, 8))
bar = sns.barplot(x='key_row', y='median_household_income', data=top_income, palette='viridis')
plt.title('Top 10 States by Median Household Income', fontsize=16, pad=20)
plt.xlabel('State', fontsize=12)
plt.ylabel('Median Household Income ($)', fontsize=12)
plt.xticks(rotation=45, ha='right')

# Add value labels on top of bars
for i, p in enumerate(bar.patches):
    height = p.get_height()
    bar.text(p.get_x() + p.get_width()/2.,
            height + 1000,
            f'${int(height):,}',
            ha="center", fontsize=9)

plt.tight_layout()
plt.savefig('visualizations/top_10_income.png', dpi=300, bbox_inches='tight')
plt.close()

# 3.4 Top 10 States by Median Sale Price
print("Creating home price bar chart...")
top_price = analysis_df.sort_values('median_sale_price', ascending=False).head(10)
plt.figure(figsize=(12, 8))
bar = sns.barplot(x='key_row', y='median_sale_price', data=top_price, palette='viridis')
plt.title('Top 10 States by Median Home Sale Price', fontsize=16, pad=20)
plt.xlabel('State', fontsize=12)
plt.ylabel('Median Sale Price ($)', fontsize=12)
plt.xticks(rotation=45, ha='right')

# Add value labels on top of bars
for i, p in enumerate(bar.patches):
    height = p.get_height()
    bar.text(p.get_x() + p.get_width()/2.,
            height + 10000,
            f'${int(height):,}',
            ha="center", fontsize=9)

plt.tight_layout()
plt.savefig('visualizations/top_10_price.png', dpi=300, bbox_inches='tight')
plt.close()

# 3.5 Top and Bottom 10 States by House Affordability Ratio
print("Creating affordability bar chart...")
affordability_high = analysis_df.sort_values('house_affordability_ratio', ascending=False).head(10)
affordability_low = analysis_df.sort_values('house_affordability_ratio').head(10)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

sns.barplot(x='key_row', y='house_affordability_ratio', data=affordability_high, palette='Reds_r', ax=ax1)
ax1.set_title('10 Least Affordable States\n(Higher Ratio = Less Affordable)', fontsize=14, pad=20)
ax1.set_xlabel('State', fontsize=12)
ax1.set_ylabel('House Affordability Ratio', fontsize=12)
ax1.tick_params(axis='x', rotation=45, labelsize=10)

sns.barplot(x='key_row', y='house_affordability_ratio', data=affordability_low, palette='Greens', ax=ax2)
ax2.set_title('10 Most Affordable States\n(Lower Ratio = More Affordable)', fontsize=14, pad=20)
ax2.set_xlabel('State', fontsize=12)
ax2.set_ylabel('House Affordability Ratio', fontsize=12)
ax2.tick_params(axis='x', rotation=45, labelsize=10)

# Add value labels
for ax, is_high in [(ax1, True), (ax2, False)]:
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.1f}',
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'bottom',
                   xytext = (0, 5), textcoords = 'offset points')

plt.suptitle('Housing Affordability Across States', fontsize=16, y=1.05)
plt.tight_layout()
plt.savefig('visualizations/affordability_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

# 3.6 Scatter Plot: Income vs. Home Prices with Affordability Ratio
print("Creating scatter plot of income vs. prices...")
plt.figure(figsize=(12, 10))
scatter = sns.scatterplot(
    x='median_household_income', 
    y='median_sale_price',
    size='census_population',
    sizes=(100, 2000),
    hue='house_affordability_ratio',
    palette='coolwarm',
    data=analysis_df
)

# Add state labels to points
for i, row in analysis_df.iterrows():
    if pd.notnull(row['state_abbrev']):
        plt.text(row['median_household_income'] + 1000, 
                row['median_sale_price'] + 5000, 
                row['state_abbrev'], 
                fontsize=9)

plt.title('Relationship Between Income and Home Prices', fontsize=16, pad=20)
plt.xlabel('Median Household Income ($)', fontsize=12)
plt.ylabel('Median Home Sale Price ($)', fontsize=12)
plt.ticklabel_format(style='plain', axis='both')
plt.tight_layout()
plt.savefig('visualizations/income_vs_price.png', dpi=300, bbox_inches='tight')
plt.close()

# Print message about skipping the map visualization
print("Skipping US map visualization since geopandas is not installed.")
print("To create map visualizations, install geopandas with: pip install geopandas")

# 3.7 Distribution plots for key metrics
print("Creating distribution plots...")
fig, axs = plt.subplots(2, 2, figsize=(16, 12))

# Population distribution
sns.histplot(analysis_df['census_population'].dropna(), kde=True, ax=axs[0, 0], color='skyblue')
axs[0, 0].set_title('Population Distribution', fontsize=12)
axs[0, 0].set_xlabel('Population', fontsize=10)
axs[0, 0].ticklabel_format(style='plain')

# Income distribution
sns.histplot(analysis_df['median_household_income'].dropna(), kde=True, ax=axs[0, 1], color='forestgreen')
axs[0, 1].set_title('Median Household Income Distribution', fontsize=12)
axs[0, 1].set_xlabel('Income ($)', fontsize=10)
axs[0, 1].ticklabel_format(style='plain')

# Home price distribution
sns.histplot(analysis_df['median_sale_price'].dropna(), kde=True, ax=axs[1, 0], color='orangered')
axs[1, 0].set_title('Median Home Sale Price Distribution', fontsize=12)
axs[1, 0].set_xlabel('Price ($)', fontsize=10)
axs[1, 0].ticklabel_format(style='plain')

# Affordability ratio distribution
sns.histplot(analysis_df['house_affordability_ratio'].dropna(), kde=True, ax=axs[1, 1], color='purple')
axs[1, 1].set_title('House Affordability Ratio Distribution', fontsize=12)
axs[1, 1].set_xlabel('Ratio (Price/Income)', fontsize=10)

plt.suptitle('Distributions of Key Metrics Across States', fontsize=16, y=0.98)
plt.tight_layout()
plt.savefig('visualizations/distributions.png', dpi=300, bbox_inches='tight')
plt.close()

# 4. Advanced Analysis

# 4.1 Z-score analysis to identify outliers
print("Performing Z-score analysis...")
z_score_columns = ['census_population', 'median_household_income', 'median_sale_price', 'house_affordability_ratio']
z_scores = pd.DataFrame()

for column in z_score_columns:
    z_scores[f'{column}_zscore'] = stats.zscore(analysis_df[column].dropna(), nan_policy='omit')

# Merge z-scores back with state data
z_scores['key_row'] = analysis_df['key_row']
outliers_df = z_scores.melt(id_vars=['key_row'], var_name='metric', value_name='z_score')
outliers_df = outliers_df[outliers_df['z_score'].abs() > 2]  # Z-score threshold of 2

# Save outliers to a separate file
if not outliers_df.empty:
    outliers_df.to_csv('visualizations/statistical_outliers.csv', index=False)

    # Create a visualization of outliers
    plt.figure(figsize=(14, 8))
    sns.barplot(x='key_row', y='z_score', hue='metric', data=outliers_df)
    plt.title('Statistical Outliers by State (|Z-Score| > 2)', fontsize=16, pad=20)
    plt.xlabel('State', fontsize=12)
    plt.ylabel('Z-Score', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Metric', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig('visualizations/outliers.png', dpi=300, bbox_inches='tight')
    plt.close()

# 4.2 Create an executive summary document
print("Creating executive summary...")
with open('visualizations/executive_summary.md', 'w') as f:
    f.write("# State Housing Affordability Analysis\n\n")
    f.write("## Executive Summary\n\n")
    
    # Overall statistics
    f.write("### Key Findings\n\n")
    f.write(f"- **Total States Analyzed**: {len(analysis_df)}\n")
    
    # Population insights
    max_pop_state = analysis_df.loc[analysis_df['census_population'].idxmax()]
    min_pop_state = analysis_df.loc[analysis_df['census_population'].idxmin()]
    f.write(f"- **Most Populous State**: {max_pop_state['key_row'].title()} ({int(max_pop_state['census_population']):,} residents)\n")
    f.write(f"- **Least Populous State**: {min_pop_state['key_row'].title()} ({int(min_pop_state['census_population']):,} residents)\n")
    
    # Income insights
    max_income_state = analysis_df.loc[analysis_df['median_household_income'].idxmax()]
    min_income_state = analysis_df.loc[analysis_df['median_household_income'].idxmin()]
    income_gap = max_income_state['median_household_income'] - min_income_state['median_household_income']
    f.write(f"- **Highest Income State**: {max_income_state['key_row'].title()} (${int(max_income_state['median_household_income']):,})\n")
    f.write(f"- **Lowest Income State**: {min_income_state['key_row'].title()} (${int(min_income_state['median_household_income']):,})\n")
    f.write(f"- **Income Gap**: ${int(income_gap):,} between highest and lowest states\n")
    
    # Housing price insights
    max_price_state = analysis_df.loc[analysis_df['median_sale_price'].idxmax()]
    min_price_state = analysis_df.loc[analysis_df['median_sale_price'].idxmin()]
    price_gap = max_price_state['median_sale_price'] - min_price_state['median_sale_price']
    f.write(f"- **Most Expensive Housing**: {max_price_state['key_row'].title()} (${int(max_price_state['median_sale_price']):,})\n")
    f.write(f"- **Least Expensive Housing**: {min_price_state['key_row'].title()} (${int(min_price_state['median_sale_price']):,})\n")
    f.write(f"- **Price Gap**: ${int(price_gap):,} between highest and lowest states\n")
    
    # Affordability insights
    max_ratio_state = analysis_df.loc[analysis_df['house_affordability_ratio'].idxmax()]
    min_ratio_state = analysis_df.loc[analysis_df['house_affordability_ratio'].idxmin()]
    f.write(f"- **Least Affordable State**: {max_ratio_state['key_row'].title()} (Ratio: {max_ratio_state['house_affordability_ratio']:.1f})\n")
    f.write(f"- **Most Affordable State**: {min_ratio_state['key_row'].title()} (Ratio: {min_ratio_state['house_affordability_ratio']:.1f})\n\n")
    
    # Relationship findings
    income_price_corr = correlation_matrix.loc['median_household_income', 'median_sale_price']
    f.write("### Relationships Between Metrics\n\n")
    f.write(f"- **Income-Housing Price Correlation**: {income_price_corr:.2f}\n")
    f.write(f"- **Affordability-Population Correlation**: {correlation_matrix.loc['house_affordability_ratio', 'census_population']:.2f}\n\n")
    
    # Recommendations section
    f.write("### Recommendations\n\n")
    f.write("1. **Monitor Affordability Trends**: States with high affordability ratios should implement policies to address housing affordability.\n")
    f.write("2. **Housing Supply Analysis**: Further analysis should examine housing supply in states with high median prices.\n")
    f.write("3. **Regional Comparisons**: Consider regional economic factors that influence housing prices relative to income.\n")
    f.write("4. **Policy Implications**: Income growth policies should be considered alongside housing market interventions.\n\n")
    
    f.write("## Visualization Directory\n\n")
    f.write("- `correlation_heatmap.png`: Shows relationships between key metrics\n")
    f.write("- `top_10_population.png`: Bar chart of most populous states\n")
    f.write("- `top_10_income.png`: Bar chart of highest income states\n")
    f.write("- `top_10_price.png`: Bar chart of states with highest home prices\n")
    f.write("- `affordability_comparison.png`: Comparison of most and least affordable states\n")
    f.write("- `income_vs_price.png`: Scatter plot showing relationship between income and home prices\n")
    f.write("- `distributions.png`: Distribution plots for key metrics\n")
    if not outliers_df.empty:
        f.write("- `outliers.png`: Bar chart showing statistical outliers\n")
        f.write("- `statistical_outliers.csv`: List of statistical outliers in the dataset\n")
    
print("Analysis complete! Visualizations saved to the 'visualizations' directory.")

Loading data for analysis...
Generating summary statistics...
       census_population  median_household_income  median_sale_price  \
count       5.200000e+01                52.000000          52.000000   
mean        6.502319e+06             77173.403846      393230.769231   
std         7.401875e+06             14810.725201      141318.388322   
min         5.840570e+05             25096.000000      138000.000000   
25%         1.916062e+06             69848.000000      282250.000000   
50%         4.379756e+06             75273.000000      374000.000000   
75%         7.526728e+06             88001.000000      483250.000000   
max         3.896519e+07            106287.000000      833000.000000   

       house_affordability_ratio  
count                  52.000000  
mean                    5.028846  
std                     1.213702  
min                     3.100000  
25%                     4.100000  
50%                     4.900000  
75%                     5.825000  
max      

/var/folders/6w/lzfylpk94j346r65d_hbhmhc0000gn/T/ipykernel_40870/869822745.py:70: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  bar = sns.barplot(x='key_row', y='census_population', data=top_pop, palette='viridis')


Creating income bar chart...


/var/folders/6w/lzfylpk94j346r65d_hbhmhc0000gn/T/ipykernel_40870/869822745.py:93: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  bar = sns.barplot(x='key_row', y='median_household_income', data=top_income, palette='viridis')


Creating home price bar chart...


/var/folders/6w/lzfylpk94j346r65d_hbhmhc0000gn/T/ipykernel_40870/869822745.py:115: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  bar = sns.barplot(x='key_row', y='median_sale_price', data=top_price, palette='viridis')


Creating affordability bar chart...


/var/folders/6w/lzfylpk94j346r65d_hbhmhc0000gn/T/ipykernel_40870/869822745.py:140: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='key_row', y='house_affordability_ratio', data=affordability_high, palette='Reds_r', ax=ax1)
/var/folders/6w/lzfylpk94j346r65d_hbhmhc0000gn/T/ipykernel_40870/869822745.py:146: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='key_row', y='house_affordability_ratio', data=affordability_low, palette='Greens', ax=ax2)


Creating scatter plot of income vs. prices...
Skipping US map visualization since geopandas is not installed.
To create map visualizations, install geopandas with: pip install geopandas
Creating distribution plots...
Performing Z-score analysis...
Creating executive summary...
Analysis complete! Visualizations saved to the 'visualizations' directory.
